# **Automatation of the CLI commands of the Snakemake workflow**

* Environment to use: [Greference_tools](code/environments/Greference_tools.yml)

In [1]:
## Libreries
import subprocess
import os
import re

In [2]:
## lists for the loops
chr_list        = ["chr3", "chr5", "chr7", "chr12", "chr17"]

url_list        = ["chromosome.3.fa.gz", "chromosome.5.fa.gz", "chromosome.7.fa.gz", 
                   "chromosome.12.fa.gz", "chromosome.17.fa.gz"]

gene_list       = ["PIK3CA", "APC", "BRAF", "KRAS", "TP53"]

sample_list     = ["ERR696683", "ERR753368", "ERR753369" ,"ERR753370" ,"ERR753371" ,"ERR753372", 
                  "ERR753373", "ERR753374", "ERR753375", "ERR753376", "ERR753377", "ERR753378"]


1. **rule download_data**

In [ ]:
subprocess.run(["snakemake", "--cores", "1", "download_data"])

* rule pre_processing

Filtering the chromosomes, make sure that the chromosomes of the config file are **ch3**. 

This rule is preatty long too... good luck

When it finished, it is recomended to delete the SAM files not oredered.

In [ ]:
## Running the 3º chromosome
subprocess.run(["snakemake", "--cores", "1", "data/original_bam/filtering/ERR696683_chr3_sorted.bam"])

## Running the rest of the chromosomes
for chr in range(len(chr_list) - 1):
    chr1 = chr_list[chr]
    chr2 = chr_list[chr + 1]
    
    print("""
          Chromosomes transformed:
          """, chr1, chr2)
    
    subprocess.run(["sed", "-i", "6,+12s/" + chr1 + "/" + chr2 + "/g", "config.yaml"])
    subprocess.run(["sed", "-i", "24s/" + chr1 + "/" + chr2 + "/g", "config.yaml"])

    subprocess.run(["snakemake", "--cores", "1", "data/original_bam/filtering/ERR696683_" + chr2 + "_sorted.bam"])

In [8]:
## reset config 
subprocess.run(["sed", "-i", "6,+12s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i", "24s/chr17/chr3/g", "config.yaml"])

CompletedProcess(args=['sed', '-i', '24s/chr17/chr3/g', 'config.yaml'], returncode=0)

* rules reference_genome; fastqc; fastp; fastqc_trimmed; bwa_mapping; sam_to_bam; delete_duplicates; extracting_variants; vep_install; vep_cli; parsing_dataR

In [9]:
## Running the workflow for the 3º chormosome

subprocess.run(["snakemake", "--cores", "1", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr3_1_fastqc.html",   
                    "data/processed/" + sample + "_chr3_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr3_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr3_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr3_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr3_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr3.vcf"])   

## Downloading VEP
subprocess.run(["snakemake", "--cores", "1", "--use-conda", "vep_install_db"])
## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr3.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Fri Aug 18 14:55:46 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Missing output files: data/reference/genome.fa
    resources: tmpdir=/tmp

rm: no se puede borrar 'data/reference/genome.fa*': No existe el archivo o el directorio
--2023-08-18 14:55:46--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.3.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 

==> If you hadn't a previous reference in the directory and there is an ERROR, is NORMAL <==



193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 59761837 (57M) [application/x-gzip]
Guardando como: ‘data/reference/genome.fa.gz’

     0K .......... .......... .......... .......... ..........  0%  458K 2m7s
    50K .......... .......... .......... .......... ..........  0% 4,25M 70s
   100K .......... .......... .......... .......... ..........  0% 1018K 66s
   150K .......... .......... .......... .......... ..........  0% 91,1M 50s
   200K .......... .......... .......... .......... ..........  0%  896K 53s
   250K .......... .......... .......... .......... ..........  0% 80,4M 44s
   300K .......... .......... .......... .......... ..........  0% 90,8M 38s
   350K .......... .......... .......... .......... ..........  0% 79,6M 33s
   400K .......... .......... .......... .......... ..........  0% 6,10M 30s
   450K .......... .......... .......... .......... ..........

application/gzip


Started analysis of ERR696683_chr3_1.fastq.gz
Approx 5% complete for ERR696683_chr3_1.fastq.gz
Approx 10% complete for ERR696683_chr3_1.fastq.gz
Approx 15% complete for ERR696683_chr3_1.fastq.gz
Approx 20% complete for ERR696683_chr3_1.fastq.gz
Approx 25% complete for ERR696683_chr3_1.fastq.gz
Approx 30% complete for ERR696683_chr3_1.fastq.gz
Approx 35% complete for ERR696683_chr3_1.fastq.gz
Approx 40% complete for ERR696683_chr3_1.fastq.gz
Approx 45% complete for ERR696683_chr3_1.fastq.gz
Approx 50% complete for ERR696683_chr3_1.fastq.gz
Approx 55% complete for ERR696683_chr3_1.fastq.gz
Approx 60% complete for ERR696683_chr3_1.fastq.gz
Approx 65% complete for ERR696683_chr3_1.fastq.gz
Approx 70% complete for ERR696683_chr3_1.fastq.gz
Approx 75% complete for ERR696683_chr3_1.fastq.gz
Approx 80% complete for ERR696683_chr3_1.fastq.gz
Approx 85% complete for ERR696683_chr3_1.fastq.gz
Approx 90% complete for ERR696683_chr3_1.fastq.gz
Approx 95% complete for ERR696683_chr3_1.fastq.gz


...


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 8 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 3_179226200_A/ACTTGA  3:1792262… ACTTGA ENSG… ENST00… Transcript   intron_var…
2 3_179226200_A/ACTTGA  3:1792262… ACTTGA ENSG… ENST00… Transcript   intron_var…
3 3_179226200_A/ACTTGA  3:1792262… ACTTGA ENSG… ENST00… Transcript   intron_var…
4 3_179226200_A/ACTTGA  3:1792262… ACTTGA ENSG… ENST00… Transcript   intron_var…
5 3_179226200_A/ACTTGA  3:1792262… ACTTGA ENSG… ENST00… Transcript   intron_var…
6 3_179226200_A/ACTTGA  3:1792262… ACTTGA ENSG… ENST00… Transcript   intron_var…
# ℹ 2 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_select


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Fri Aug 18 17:02:34 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-18T170230.070985.snakemake.log


# A tibble: 86 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   intron_var…
2 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   downstream…
3 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   downstream…
4 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   intron_var…
5 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   upstream_g…
6 3_179199217_A/G       3:1791992… G      ENSG… ENST00… Transcript   intron_var…
# ℹ 80 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele

CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

## Changing the config file to do the chromosome 5

In [10]:
subprocess.run(["sed", "-i", "6,+12s/chr3/chr5/g", "config.yaml"])
subprocess.run(["sed", "-i","24s/chr3/chr5/g", "config.yaml"])
subprocess.run(["sed", "-i", "31s/chromosome.3.fa.gz/chromosome.5.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "50s/PIK3CA/APC/g", "config.yaml"])


CompletedProcess(args=['sed', '-i', '50s/PIK3CA/APC/g', 'config.yaml'], returncode=0)

In [11]:
## Running the workflow for the 5º chormosome

subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr5_1_fastqc.html",   
                    "data/processed/" + sample + "_chr5_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr5_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr5_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr5_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr5_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr5.vcf"])   

## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr5.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Fri Aug 18 17:02:54 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Forced execution
    resources: tmpdir=/tmp

--2023-08-18 17:02:54--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.5.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 54485176 (52M) [application/x-gzip]
Guardando como: ‘data

application/gzip


Started analysis of ERR696683_chr5_1_fastp.fastq.gz
Approx 5% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 10% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 15% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 20% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 25% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 30% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 35% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 40% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 45% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 50% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 55% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 60% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 65% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 70% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 75% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 80% complete for ERR696683_chr5_1_fastp.fastq.gz
Approx 85% complete for ERR696683_chr5_1_fastp.fastq.

Analysis complete for ERR696683_chr5_1_fastp.fastq.gz
application/gzip


Started analysis of ERR696683_chr5_2_fastp.fastq.gz
Approx 5% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 10% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 15% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 20% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 25% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 30% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 35% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 40% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 45% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 50% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 55% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 60% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 65% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 70% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 75% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 80% complete for ERR696683_chr5_2_fastp.fastq.gz
Approx 85% complete for ERR696683_chr5_2_fastp.fastq.

...

# A tibble: 59 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 5_112729129_G/C       5:1127291… C      ENSG… ENST00… Transcript   intron_var…
2 5_112729129_G/C       5:1127291… C      ENSG… ENST00… Transcript   intron_var…
3 5_112729129_G/C       5:1127291… C      ENSG… ENST00… Transcript   intron_var…
4 5_112775373_G/A       5:1127753… A      ENSG… ENST00… Transcript   intron_var…
5 5_112775373_G/A       5:1127753… A      ENSG… ENST00… Transcript   intron_var…
6 5_112775373_G/A       5:1127753… A      ENSG… ENST00… Transcript   intron_var…
# ℹ 53 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 71 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
2 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
3 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
4 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
5 5_112777830_A/T       5:1127778… T      ENSG… ENST00… Transcript   intron_var…
6 5_112778038_C/T       5:1127780… T      ENSG… ENST00… Transcript   intron_var…
# ℹ 65 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Fri Aug 18 18:29:55 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-18T182951.504585.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

## Changing the config file to do the chromosome 7 

In [3]:
subprocess.run(["sed", "-i", "6,+12s/chr5/chr7/g", "config.yaml"])
subprocess.run(["sed", "-i","24s/chr5/chr7/g", "config.yaml"])
subprocess.run(["sed", "-i", "31s/chromosome.5.fa.gz/chromosome.7.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "50s/APC/BRAF/g", "config.yaml"])


CompletedProcess(args=['sed', '-i', '50s/APC/BRAF/g', 'config.yaml'], returncode=0)

In [4]:
## Running the workflow for the 7º chormosome

subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr7_1_fastqc.html",   
                    "data/processed/" + sample + "_chr7_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr7_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr7_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr7_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr7_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr7.vcf"])   

## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr7.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Fri Aug 18 20:45:27 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Forced execution
    resources: tmpdir=/tmp

--2023-08-18 20:45:27--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.7.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 47416846 (45M) [application/x-gzip]
Guardando como: ‘data

application/gzip


Started analysis of ERR696683_chr7_1.fastq.gz
Approx 5% complete for ERR696683_chr7_1.fastq.gz
Approx 10% complete for ERR696683_chr7_1.fastq.gz
Approx 15% complete for ERR696683_chr7_1.fastq.gz
Approx 20% complete for ERR696683_chr7_1.fastq.gz
Approx 25% complete for ERR696683_chr7_1.fastq.gz
Approx 30% complete for ERR696683_chr7_1.fastq.gz
Approx 35% complete for ERR696683_chr7_1.fastq.gz
Approx 40% complete for ERR696683_chr7_1.fastq.gz
Approx 45% complete for ERR696683_chr7_1.fastq.gz
Approx 50% complete for ERR696683_chr7_1.fastq.gz
Approx 55% complete for ERR696683_chr7_1.fastq.gz
Approx 60% complete for ERR696683_chr7_1.fastq.gz
Approx 65% complete for ERR696683_chr7_1.fastq.gz
Approx 70% complete for ERR696683_chr7_1.fastq.gz
Approx 75% complete for ERR696683_chr7_1.fastq.gz
Approx 80% complete for ERR696683_chr7_1.fastq.gz
Approx 85% complete for ERR696683_chr7_1.fastq.gz
Approx 90% complete for ERR696683_chr7_1.fastq.gz
Approx 95% complete for ERR696683_chr7_1.fastq.gz


Analysis complete for ERR696683_chr7_1.fastq.gz
application/gzip


Started analysis of ERR696683_chr7_2.fastq.gz
Approx 5% complete for ERR696683_chr7_2.fastq.gz
Approx 10% complete for ERR696683_chr7_2.fastq.gz
Approx 15% complete for ERR696683_chr7_2.fastq.gz
Approx 20% complete for ERR696683_chr7_2.fastq.gz
Approx 25% complete for ERR696683_chr7_2.fastq.gz
Approx 30% complete for ERR696683_chr7_2.fastq.gz
Approx 35% complete for ERR696683_chr7_2.fastq.gz
Approx 40% complete for ERR696683_chr7_2.fastq.gz
Approx 45% complete for ERR696683_chr7_2.fastq.gz
Approx 50% complete for ERR696683_chr7_2.fastq.gz
Approx 55% complete for ERR696683_chr7_2.fastq.gz
Approx 60% complete for ERR696683_chr7_2.fastq.gz
Approx 65% complete for ERR696683_chr7_2.fastq.gz
Approx 70% complete for ERR696683_chr7_2.fastq.gz
Approx 75% complete for ERR696683_chr7_2.fastq.gz
Approx 80% complete for ERR696683_chr7_2.fastq.gz
Approx 85% complete for ERR696683_chr7_2.fastq.gz
Approx 90% complete for ERR696683_chr7_2.fastq.gz
Approx 95% complete for ERR696683_chr7_2.fastq.gz


...

# A tibble: 44 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 7_140726457_A/G       7:1407264… G      ENSG… ENST00… Transcript   3_prime_UT…
2 7_140726457_A/G       7:1407264… G      ENSG… ENST00… Transcript   non_coding…
3 7_140726457_A/G       7:1407264… G      ENSG… ENST00… Transcript   non_coding…
4 7_140726457_A/G       7:1407264… G      ENSG… ENST00… Transcript   3_prime_UT…
5 7_140726457_A/G       7:1407264… G      ENSG… ENST00… Transcript   downstream…
6 7_140749271_C/G       7:1407492… G      ENSG… ENST00… Transcript   intron_var…
# ℹ 38 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 0 × 44
# ℹ 44 variables: #uploaded_variation <chr>, location <chr>, allele <chr>,
#   gene <chr>, feature <chr>, feature_type <chr>, consequence <chr>,
#   cdna_position <chr>, cds_position <chr>, protein_position <chr>,
#   amino_acids <chr>, codons <chr>, existing_variation <chr>, impact <chr>,
#   distance <chr>, strand <chr>, flags <chr>, variant_class <chr>,
#   symbol <chr>, symbol_source <chr>, hgnc_id <chr>, biotype <chr>,
#   mane_select <chr>, mane_plus_clinical <chr>, tsl <chr>, appris <chr>, …
# A tibble: 26 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   synonymous…
2 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   synonymous…
3 7_140749350_T/C       7:1407493… C      ENSG… ENST00… Transcript   synonymous…
4 7_140749350_T/C       7:1407493… C      ENSG… ENST


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Fri Aug 18 22:16:35 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-18T221631.141305.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

## Changing the config file to do the chromosome 12 

In [5]:
subprocess.run(["sed", "-i", "6,+12s/chr7/chr12/g", "config.yaml"])
subprocess.run(["sed", "-i","24s/chr7/chr12/g", "config.yaml"])
subprocess.run(["sed", "-i", "31s/chromosome.7.fa.gz/chromosome.12.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "50s/BRAF/KRAS/g", "config.yaml"])


CompletedProcess(args=['sed', '-i', '50s/BRAF/KRAS/g', 'config.yaml'], returncode=0)

In [6]:
## Running the workflow for the 12º chormosome

subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr12_1_fastqc.html",   
                    "data/processed/" + sample + "_chr12_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr12_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr12_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr12_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr12_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr12.vcf"])   

## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr12.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Fri Aug 18 22:53:29 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Forced execution
    resources: tmpdir=/tmp

--2023-08-18 22:53:29--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 39795542 (38M) [application/x-gzip]
Guardando como: ‘dat

application/gzip


Started analysis of ERR696683_chr12_1.fastq.gz
Approx 5% complete for ERR696683_chr12_1.fastq.gz
Approx 10% complete for ERR696683_chr12_1.fastq.gz
Approx 15% complete for ERR696683_chr12_1.fastq.gz
Approx 20% complete for ERR696683_chr12_1.fastq.gz
Approx 25% complete for ERR696683_chr12_1.fastq.gz
Approx 30% complete for ERR696683_chr12_1.fastq.gz
Approx 35% complete for ERR696683_chr12_1.fastq.gz
Approx 40% complete for ERR696683_chr12_1.fastq.gz
Approx 45% complete for ERR696683_chr12_1.fastq.gz
Approx 50% complete for ERR696683_chr12_1.fastq.gz
Approx 55% complete for ERR696683_chr12_1.fastq.gz
Approx 60% complete for ERR696683_chr12_1.fastq.gz
Approx 65% complete for ERR696683_chr12_1.fastq.gz
Approx 70% complete for ERR696683_chr12_1.fastq.gz
Approx 75% complete for ERR696683_chr12_1.fastq.gz
Approx 80% complete for ERR696683_chr12_1.fastq.gz
Approx 85% complete for ERR696683_chr12_1.fastq.gz
Approx 90% complete for ERR696683_chr12_1.fastq.gz
Approx 95% complete for ERR696683_ch

Analysis complete for ERR696683_chr12_1.fastq.gz
application/gzip


Started analysis of ERR696683_chr12_2.fastq.gz
Approx 5% complete for ERR696683_chr12_2.fastq.gz
Approx 10% complete for ERR696683_chr12_2.fastq.gz
Approx 15% complete for ERR696683_chr12_2.fastq.gz
Approx 20% complete for ERR696683_chr12_2.fastq.gz
Approx 25% complete for ERR696683_chr12_2.fastq.gz
Approx 30% complete for ERR696683_chr12_2.fastq.gz
Approx 35% complete for ERR696683_chr12_2.fastq.gz
Approx 40% complete for ERR696683_chr12_2.fastq.gz
Approx 45% complete for ERR696683_chr12_2.fastq.gz
Approx 50% complete for ERR696683_chr12_2.fastq.gz
Approx 55% complete for ERR696683_chr12_2.fastq.gz
Approx 60% complete for ERR696683_chr12_2.fastq.gz
Approx 65% complete for ERR696683_chr12_2.fastq.gz
Approx 70% complete for ERR696683_chr12_2.fastq.gz
Approx 75% complete for ERR696683_chr12_2.fastq.gz
Approx 80% complete for ERR696683_chr12_2.fastq.gz
Approx 85% complete for ERR696683_chr12_2.fastq.gz
Approx 90% complete for ERR696683_chr12_2.fastq.gz
Approx 95% complete for ERR696683_ch

...

# A tibble: 12 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 12_25209618_A/C       12:252096… C      ENSG… ENST00… Transcript   3_prime_UT…
2 12_25209618_A/C       12:252096… C      ENSG… ENST00… Transcript   3_prime_UT…
3 12_25209618_A/C       12:252096… C      ENSG… ENST00… Transcript   3_prime_UT…
4 12_25209618_A/C       12:252096… C      ENSG… ENST00… Transcript   3_prime_UT…
5 12_25209618_A/C       12:252096… C      ENSG… ENST00… Transcript   3_prime_UT…
6 12_25209618_A/C       12:252096… C      ENSG… ENST00… Transcript   3_prime_UT…
# ℹ 6 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_selec


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 12 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
2 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
3 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
4 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   synonymous…
5 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
6 12_25209843_A/G       12:252098… G      ENSG… ENST00… Transcript   3_prime_UT…
# ℹ 6 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_selec


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.

Touching output file tasks/13parsing_dataR.done.
[Sat Aug 19 00:12:47 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-19T001243.679343.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

## Changing the config file to do the chromosome 17 

In [7]:
subprocess.run(["sed", "-i", "6,+12s/chr12/chr17/g", "config.yaml"])
subprocess.run(["sed", "-i","24s/chr12/chr17/g", "config.yaml"])
subprocess.run(["sed", "-i", "31s/chromosome.12.fa.gz/chromosome.17.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "50s/KRAS/TP53/g", "config.yaml"])


CompletedProcess(args=['sed', '-i', '50s/KRAS/TP53/g', 'config.yaml'], returncode=0)

In [8]:
## Running the workflow for the 17º chormosome

subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) ## rule reference_genome 

for sample in sample_list:
    ## Quality inspection
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/fastqc_result/" + sample + "_chr17_1_fastqc.html",   
                    "data/processed/" + sample + "_chr17_1_fastp.fastq.gz",   
                    "results/fastqc_result/trimmed/" + sample + "_chr17_1_fastp_fastqc.html"])
    
for sample in sample_list:
    ## Mapping reads
    subprocess.run(["snakemake", "--cores", "1", "--use-conda",
                    "results/mapped_reads/" + sample + "_chr17_sorted.sam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr17_sorted.bam",   
                    "results/mapped_reads/bam_files/" + sample + "_chr17_dedup.bam"])
    
for sample in sample_list:
    # Extracting variants  
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/" + sample + "_chr17.vcf"])   

## VEP CLI
for sample in sample_list:
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "results/variants/vep/" + sample + "_chr17.txt"])

## Extracting the gene PIKCA3 from the VEP files
subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Sat Aug 19 00:13:23 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Forced execution
    resources: tmpdir=/tmp

--2023-08-19 00:13:23--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.17.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 24104573 (23M) [application/x-gzip]
Guardando como: ‘dat

application/gzip


Started analysis of ERR696683_chr17_1.fastq.gz
Approx 5% complete for ERR696683_chr17_1.fastq.gz
Approx 10% complete for ERR696683_chr17_1.fastq.gz
Approx 15% complete for ERR696683_chr17_1.fastq.gz
Approx 20% complete for ERR696683_chr17_1.fastq.gz
Approx 25% complete for ERR696683_chr17_1.fastq.gz
Approx 30% complete for ERR696683_chr17_1.fastq.gz
Approx 35% complete for ERR696683_chr17_1.fastq.gz
Approx 40% complete for ERR696683_chr17_1.fastq.gz
Approx 45% complete for ERR696683_chr17_1.fastq.gz
Approx 50% complete for ERR696683_chr17_1.fastq.gz
Approx 55% complete for ERR696683_chr17_1.fastq.gz
Approx 60% complete for ERR696683_chr17_1.fastq.gz
Approx 65% complete for ERR696683_chr17_1.fastq.gz
Approx 70% complete for ERR696683_chr17_1.fastq.gz
Approx 75% complete for ERR696683_chr17_1.fastq.gz
Approx 80% complete for ERR696683_chr17_1.fastq.gz
Approx 85% complete for ERR696683_chr17_1.fastq.gz
Approx 90% complete for ERR696683_chr17_1.fastq.gz
Approx 95% complete for ERR696683_ch

Analysis complete for ERR696683_chr17_1.fastq.gz
application/gzip


Started analysis of ERR696683_chr17_2.fastq.gz
Approx 5% complete for ERR696683_chr17_2.fastq.gz
Approx 10% complete for ERR696683_chr17_2.fastq.gz
Approx 15% complete for ERR696683_chr17_2.fastq.gz
Approx 20% complete for ERR696683_chr17_2.fastq.gz
Approx 25% complete for ERR696683_chr17_2.fastq.gz
Approx 30% complete for ERR696683_chr17_2.fastq.gz
Approx 35% complete for ERR696683_chr17_2.fastq.gz
Approx 40% complete for ERR696683_chr17_2.fastq.gz
Approx 45% complete for ERR696683_chr17_2.fastq.gz
Approx 50% complete for ERR696683_chr17_2.fastq.gz
Approx 55% complete for ERR696683_chr17_2.fastq.gz
Approx 60% complete for ERR696683_chr17_2.fastq.gz
Approx 65% complete for ERR696683_chr17_2.fastq.gz
Approx 70% complete for ERR696683_chr17_2.fastq.gz
Approx 75% complete for ERR696683_chr17_2.fastq.gz
Approx 80% complete for ERR696683_chr17_2.fastq.gz
Approx 85% complete for ERR696683_chr17_2.fastq.gz
Approx 90% complete for ERR696683_chr17_2.fastq.gz
Approx 95% complete for ERR696683_ch

...

# A tibble: 52 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
2 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
3 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
4 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
5 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
6 17_7676154_G/C        17:7676154 C      ENSG… ENST00… Transcript   missense_v…
# ℹ 46 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_sele


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.



# A tibble: 149 × 44
  `#uploaded_variation` location   allele gene  feature feature_type consequence
  <chr>                 <chr>      <chr>  <chr> <chr>   <chr>        <chr>      
1 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   downstream…
2 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   intron_var…
3 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   intron_var…
4 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   downstream…
5 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   downstream…
6 17_7666380_A/C        17:7666380 C      ENSG… ENST00… Transcript   downstream…
# ℹ 143 more rows
# ℹ 37 more variables: cdna_position <chr>, cds_position <chr>,
#   protein_position <chr>, amino_acids <chr>, codons <chr>,
#   existing_variation <chr>, impact <chr>, distance <chr>, strand <chr>,
#   flags <chr>, variant_class <chr>, symbol <chr>, symbol_source <chr>,
#   hgnc_id <chr>, biotype <chr>, mane_se

Touching output file tasks/13parsing_dataR.done.
[Sat Aug 19 01:20:03 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-19T011959.421244.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'parsing_dataR'], returncode=0)

In [9]:
## reset config 
subprocess.run(["sed", "-i", "6,+12s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i", "24s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i", "31s/chromosome.17.fa.gz/chromosome.3.fa.gz/g", "config.yaml"])
subprocess.run(["sed", "-i", "50s/TP53/PIK3CA/g", "config.yaml"])

CompletedProcess(args=['sed', '-i', '50s/TP53/PIK3CA/g', 'config.yaml'], returncode=0)

## **Last rule**

### **rule R_plotting**

Plotting the data of the 5 gene tables

In [10]:
subprocess.run(["snakemake", "--cores", "1", "R_plotting"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job           count    min threads    max threads
----------  -------  -------------  -------------
R_plotting        1              1              1
total             1              1              1

Select jobs to execute...

[Sat Aug 19 01:25:37 2023]
rule R_plotting:
    input: code/06final_plot.R
    output: results/biostatistics/plots/final_plot.png, results/biostatistics/plots/other_plot.png
    jobid: 0
    reason: Missing output files: results/biostatistics/plots/other_plot.png, results/biostatistics/plots/final_plot.png
    resources: tmpdir=/tmp



[1] "##==========================##"
[1] "##===> WORK FINISHED!!! <===##"
[1] "##==========================##"
THE SCRIPT RAN WELL congrats :)


[Sat Aug 19 01:25:41 2023]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2023-08-19T012536.811650.snakemake.log


CompletedProcess(args=['snakemake', '--cores', '1', 'R_plotting'], returncode=0)